In [2]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [3]:
!nvidia-smi -L

GPU 0: NVIDIA RTX A6000 (UUID: GPU-7de0cdd4-72bc-b63b-c746-81bfbe469d2a)


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/gpt-j-6b",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6b")
tokenizer.pad_token = tokenizer.eos_token

In [5]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [6]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [8]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Load the datasets
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="../data/preprocessed/train_with_reasoning.csv",
    block_size=256,
)

val_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="../data/preprocessed/val_with_reasoning.csv",
    block_size=256,
)

# Define data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (6576023 > 2048). Running this sequence through the model will result in indexing errors


In [9]:
from transformers import Trainer, TrainingArguments

# Define the parameters for fine-tuning
lr = 1e-5
end_lr = 2e-6
num_train_epochs = 1
warmup_steps = 100

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir='gptj-with-reasoning-results',
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=4,
        warmup_steps=warmup_steps,
        num_train_epochs=num_train_epochs,
        learning_rate=lr,
        weight_decay=0.1,
        fp16=True,
        logging_steps=10,
        logging_dir='./logs',
        evaluation_strategy="steps",
        eval_steps=50,
        
    ),
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)
model.config.use_cache = False

In [10]:
trainer.train()
trainer.save_model()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
50,2.121100,2.118749
100,2.049500,2.041663
150,1.917400,1.908586
200,1.817600,1.815392
250,1.730000,1.755741
300,1.705000,1.724064
350,1.693100,1.703592
400,1.669900,1.688870
450,1.669300,1.678436
500,1.629300,1.670868


/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [12]:
model.push_to_hub("harsha28/gptj-lfqa-with-reasoning1",
                  use_auth_token=True,
                  commit_message="lr 1e-5, 1 epoch",
                  private=True)

adapter_model.bin:   0%|          | 0.00/29.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/harsha28/gptj-lfqa-with-reasoning/commit/f247d157f68dccb195bff389d1ce7f7b13ec4ff6', commit_message='lr 1e-5, 1 epoch', commit_description='', oid='f247d157f68dccb195bff389d1ce7f7b13ec4ff6', pr_url=None, pr_revision=None, pr_num=None)